In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

In [2]:
qc_a = QuantumCircuit(4, 4)
qc_a.iswap(0, 2)
qc_a.measure([0, 1, 2, 3], [0, 1, 2, 3])

In [3]:
import qiskit_aer
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeVigo
from qiskit import transpile

In [4]:
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())
pm2 = generate_preset_pass_manager(backend=backend, optimization_level=1)
qca_trans = pm2.run(qc_a)

In [5]:
qca_trans

In [6]:
from qiskit_draw_utils import get_layered_instructions 

In [7]:
dx = get_layered_instructions(qca_trans)

In [12]:
def getCircuitLayout(layered_instructions):
    dl = []
    dx = layered_instructions[2]
    for layer in dx:
        ol = {
            "operations": [],
            "num_op": 0
        }
        for inst in layer:
            op = {
                "gate": inst.op.name,
                "num_qubits": inst.op.num_qubits,
                "num_clbits": inst.op.num_clbits,
                "params": inst.op.params,
                "qubits": [],
                "clbits": []
            }
            for qubit in inst.qargs:
                op["qubits"].append({
                    "register": { "name": qubit._register._name, "size": qubit._register._size },
                    "index": qubit._index
                })
            op["qubits"] = tuple(op["qubits"])
            for clbit in inst.cargs:
                op["clbits"].append({
                    "register": { "name": qubit._register._name, "size": qubit._register._size },
                    "index": qubit._index
                })
            op["clbits"] = tuple(op["clbits"])
            ol["operations"].append(op)
            ol["num_op"] = ol["num_op"] + 1
        dl.append(ol)
    return dl

In [13]:
getCircuitLayout(dx)

[{'operations': [{'gate': 'rz',
    'num_qubits': 1,
    'num_clbits': 0,
    'params': [-3.141592653589793],
    'qubits': ({'register': {'name': 'q', 'size': 5}, 'index': 1},),
    'clbits': ()},
   {'gate': 'rz',
    'num_qubits': 1,
    'num_clbits': 0,
    'params': [1.5707963267948966],
    'qubits': ({'register': {'name': 'q', 'size': 5}, 'index': 2},),
    'clbits': ()},
   {'gate': 'measure',
    'num_qubits': 1,
    'num_clbits': 1,
    'params': [],
    'qubits': ({'register': {'name': 'q', 'size': 5}, 'index': 3},),
    'clbits': ({'register': {'name': 'q', 'size': 5}, 'index': 3},)}],
  'num_op': 3},
 {'operations': [{'gate': 'measure',
    'num_qubits': 1,
    'num_clbits': 1,
    'params': [],
    'qubits': ({'register': {'name': 'q', 'size': 5}, 'index': 4},),
    'clbits': ({'register': {'name': 'q', 'size': 5}, 'index': 4},)},
   {'gate': 'sx',
    'num_qubits': 1,
    'num_clbits': 0,
    'params': [],
    'qubits': ({'register': {'name': 'q', 'size': 5}, 'index': 1}

In [8]:
for layer in dx[2]:
    print(layer)
    print("----\n")

[DAGOpNode(op=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-3.141592653589793]), qargs=(Qubit(QuantumRegister(5, 'q'), 1),), cargs=()), DAGOpNode(op=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qargs=(Qubit(QuantumRegister(5, 'q'), 2),), cargs=()), DAGOpNode(op=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qargs=(Qubit(QuantumRegister(5, 'q'), 3),), cargs=(Clbit(ClassicalRegister(4, 'c'), 1),))]
----

[DAGOpNode(op=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qargs=(Qubit(QuantumRegister(5, 'q'), 4),), cargs=(Clbit(ClassicalRegister(4, 'c'), 3),)), DAGOpNode(op=Instruction(name='sx', num_qubits=1, num_clbits=0, params=[]), qargs=(Qubit(QuantumRegister(5, 'q'), 1),), cargs=())]
----

[DAGOpNode(op=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qargs=(Qubit(QuantumRegister(5, 'q'), 1),), cargs=())]
----

[DAGOpNode(op=Instruction(name='cx', num_qubits=2, num_cl

In [74]:
qca_trans.draw()

»
ancilla_0 -> 0 ──────────────────────────────────────────────────────────»
                ┌────────┐┌────┐┌─────────┐     ┌───┐           ┌─┐      »
      q_0 -> 1 ─┤ Rz(-π) ├┤ √X ├┤ Rz(π/2) ├──■──┤ X ├───────────┤M├──────»
               ┌┴────────┤└────┘└─────────┘┌─┴─┐└─┬─┘┌─────────┐└╥┘┌────┐»
      q_2 -> 2 ┤ Rz(π/2) ├─────────────────┤ X ├──■──┤ Rz(π/2) ├─╫─┤ √X ├»
               └───┬─┬───┘                 └───┘     └─────────┘ ║ └────┘»
      q_1 -> 3 ────┤M├───────────────────────────────────────────╫───────»
                   └╥┘     ┌─┐                                   ║       »
      q_3 -> 4 ─────╫──────┤M├───────────────────────────────────╫───────»
                    ║      └╥┘                                   ║       »
          c: 4/═════╩═══════╩════════════════════════════════════╩═══════»
                    1       3                                    0       »
«                             
«ancilla_0 -> 0 ──────────────
«                             
«      q_0 -> 1 ──────────────
«               ┌─────────┐┌─┐
«      q_2 -> 2 ┤ Rz(π/2) ├┤M├
«               └─────────┘└╥┘
«      q_1 -> 3 ────────────╫─
«                           ║ 
«      q_3 -> 4 ────────────╫─
«                           ║ 
«          c: 4/════════════╩═
«                           2

In [64]:
from collections import OrderedDict
from qiskit.converters import circuit_to_dag


def _get_layered_instructions(
    circuit, reverse_bits=False, justify=None, idle_wires=True, wire_order=None, wire_map=None
):
    """
    Given a circuit, return a tuple (qubits, clbits, nodes) where
    qubits and clbits are the quantum and classical registers
    in order (based on reverse_bits or wire_order) and nodes
    is a list of DAGOpNodes.

    Args:
        circuit (QuantumCircuit): From where the information is extracted.
        reverse_bits (bool): If true the order of the bits in the registers is
            reversed.
        justify (str) : `left`, `right` or `none`. Defaults to `left`. Says how
            the circuit should be justified.
        idle_wires (bool): Include idle wires. Default is True.
        wire_order (list): A list of ints that modifies the order of the bits

    Returns:
        Tuple(list,list,list): To be consumed by the visualizer directly.

    Raises:
        VisualizationError: if both reverse_bits and wire_order are entered.
    """
    if justify:
        justify = justify.lower()

    # default to left
    justify = justify if justify in ("right", "none") else "left"

    if wire_map is not None:
        qubits = [bit for bit in wire_map if isinstance(bit, Qubit)]
    else:
        qubits = circuit.qubits.copy()
    clbits = circuit.clbits.copy()
    nodes = []

    # Create a mapping of each register to the max layer number for all measure ops
    # with that register as the target. Then when an op with condition is seen,
    # it will be placed to the right of the measure op if the register matches.
    measure_map = OrderedDict([(c, -1) for c in clbits])

    if reverse_bits and wire_order is not None:
        raise VisualizationError("Cannot set both reverse_bits and wire_order in the same drawing.")

    if reverse_bits:
        qubits.reverse()
        clbits.reverse()
    elif wire_order is not None:
        new_qubits = []
        new_clbits = []
        for bit in wire_order:
            if bit < len(qubits):
                new_qubits.append(qubits[bit])
            else:
                new_clbits.append(clbits[bit - len(qubits)])
        qubits = new_qubits
        clbits = new_clbits

    dag = circuit_to_dag(circuit)
    dag.qubits = qubits
    dag.clbits = clbits

    if justify == "none":
        for node in dag.topological_op_nodes():
            nodes.append([node])
    else:
        nodes = _LayerSpooler(dag, justify, measure_map)

    # Optionally remove all idle wires and instructions that are on them and
    # on them only.
    if not idle_wires:
        for wire in dag.idle_wires(ignore=["barrier", "delay"]):
            if wire in qubits:
                qubits.remove(wire)
            if wire in clbits:
                clbits.remove(wire)

    nodes = [[node for node in layer if any(q in qubits for q in node.qargs)] for layer in nodes]

    return qubits, clbits, nodes

def _sorted_nodes(dag_layer):
    """Convert DAG layer into list of nodes sorted by node_id
    qiskit-terra #2802
    """
    nodes = dag_layer["graph"].op_nodes()
    # sort into the order they were input
    nodes.sort(key=lambda nd: nd._node_id)
    return nodes


def _get_gate_span(qubits, node):
    """Get the list of qubits drawing this gate would cover
    qiskit-terra #2802
    """
    min_index = len(qubits)
    max_index = 0
    for qreg in node.qargs:
        index = qubits.index(qreg)

        if index < min_index:
            min_index = index
        if index > max_index:
            max_index = index

    # Because of wrapping boxes for mpl control flow ops, this
    # type of op must be the only op in the layer
    if isinstance(node.op, ControlFlowOp):
        span = qubits
    elif node.cargs or getattr(node.op, "condition", None):
        span = qubits[min_index : len(qubits)]
    else:
        span = qubits[min_index : max_index + 1]

    return span



def _sorted_nodes(dag_layer):
    """Convert DAG layer into list of nodes sorted by node_id
    qiskit-terra #2802
    """
    nodes = dag_layer["graph"].op_nodes()
    # sort into the order they were input
    nodes.sort(key=lambda nd: nd._node_id)
    return nodes



class _LayerSpooler(list):
    """Manipulate list of layer dicts for _get_layered_instructions."""

    def __init__(self, dag, justification, measure_map):
        """Create spool"""
        super().__init__()
        self.dag = dag
        self.qubits = dag.qubits
        self.clbits = dag.clbits
        self.justification = justification
        self.measure_map = measure_map
        self.cregs = [self.dag.cregs[reg] for reg in self.dag.cregs]

        if self.justification == "left":
            for dag_layer in dag.layers():
                current_index = len(self) - 1
                dag_nodes = _sorted_nodes(dag_layer)
                for node in dag_nodes:
                    self.add(node, current_index)
        else:
            dag_layers = []
            for dag_layer in dag.layers():
                dag_layers.append(dag_layer)

            # going right to left!
            dag_layers.reverse()

            for dag_layer in dag_layers:
                current_index = 0
                dag_nodes = _sorted_nodes(dag_layer)
                for node in dag_nodes:
                    self.add(node, current_index)

    def is_found_in(self, node, nodes):
        """Is any qreq in node found in any of nodes?"""
        all_qargs = []
        for a_node in nodes:
            for qarg in a_node.qargs:
                all_qargs.append(qarg)
        return any(i in node.qargs for i in all_qargs)

    def insertable(self, node, nodes):
        """True .IFF. we can add 'node' to layer 'nodes'"""
        return not _any_crossover(self.qubits, node, nodes)

    def slide_from_left(self, node, index):
        """Insert node into first layer where there is no conflict going l > r"""
        measure_layer = None
        if isinstance(node.op, Measure):
            measure_bit = next(bit for bit in self.measure_map if node.cargs[0] == bit)

        if not self:
            inserted = True
            self.append([node])
        else:
            inserted = False
            curr_index = index
            last_insertable_index = -1
            index_stop = -1
            if (condition := getattr(node.op, "condition", None)) is not None:
                index_stop = max(
                    (self.measure_map[bit] for bit in condition_resources(condition).clbits),
                    default=index_stop,
                )
            if node.cargs:
                for carg in node.cargs:
                    try:
                        carg_bit = next(bit for bit in self.measure_map if carg == bit)
                        if self.measure_map[carg_bit] > index_stop:
                            index_stop = self.measure_map[carg_bit]
                    except StopIteration:
                        pass
            while curr_index > index_stop:
                if self.is_found_in(node, self[curr_index]):
                    break
                if self.insertable(node, self[curr_index]):
                    last_insertable_index = curr_index
                curr_index = curr_index - 1

            if last_insertable_index >= 0:
                inserted = True
                self[last_insertable_index].append(node)
                measure_layer = last_insertable_index
            else:
                inserted = False
                curr_index = index
                while curr_index < len(self):
                    if self.insertable(node, self[curr_index]):
                        self[curr_index].append(node)
                        measure_layer = curr_index
                        inserted = True
                        break
                    curr_index = curr_index + 1

        if not inserted:
            self.append([node])

        if isinstance(node.op, Measure):
            if not measure_layer:
                measure_layer = len(self) - 1
            if measure_layer > self.measure_map[measure_bit]:
                self.measure_map[measure_bit] = measure_layer

    def slide_from_right(self, node, index):
        """Insert node into rightmost layer as long there is no conflict."""
        if not self:
            self.insert(0, [node])
            inserted = True
        else:
            inserted = False
            curr_index = index
            last_insertable_index = None

            while curr_index < len(self):
                if self.is_found_in(node, self[curr_index]):
                    break
                if self.insertable(node, self[curr_index]):
                    last_insertable_index = curr_index
                curr_index = curr_index + 1

            if last_insertable_index:
                self[last_insertable_index].append(node)
                inserted = True
            else:
                curr_index = index
                while curr_index > -1:
                    if self.insertable(node, self[curr_index]):
                        self[curr_index].append(node)
                        inserted = True
                        break
                    curr_index = curr_index - 1

        if not inserted:
            self.insert(0, [node])

    def add(self, node, index):
        """Add 'node' where it belongs, starting the try at 'index'."""
        if self.justification == "left":
            self.slide_from_left(node, index)
        else:
            self.slide_from_right(node, index)


def _any_crossover(qubits, node, nodes):
    """Return True .IFF. 'node' crosses over any 'nodes'."""
    gate_span = _get_gate_span(qubits, node)
    all_indices = []
    for check_node in nodes:
        if check_node != node:
            all_indices += _get_gate_span(qubits, check_node)
    return any(i in gate_span for i in all_indices)



In [9]:


from __future__ import annotations

import logging
import os
import subprocess
import tempfile
import shutil
import typing
from warnings import warn

In [14]:
from qiskit import user_config
from qiskit.utils import optionals as _optionals
from qiskit.circuit import ControlFlowOp, Measure

In [21]:
def circuit_drawer(
    circuit: QuantumCircuit,
    scale: float | None = None,
    filename: str | None = None,
    style: dict | str | None = None,
    output: str | None = None,
    interactive: bool = False,
    plot_barriers: bool = True,
    reverse_bits: bool | None = None,
    justify: str | None = None,
    vertical_compression: str | None = "medium",
    idle_wires: bool = True,
    with_layout: bool = True,
    fold: int | None = None,
    # The type of ax is matplotlib.axes.Axes, but this is not a fixed dependency, so cannot be
    # safely forward-referenced.
    ax: Any | None = None,
    initial_state: bool = False,
    cregbundle: bool | None = None,
    wire_order: list[int] | None = None,
    expr_len: int = 30,
):
    image = None
    expr_len = max(expr_len, 0)
    config = user_config.get_config()
    # Get default from config file else use text
    default_output = "text"
    default_reverse_bits = False
    if config:
        default_output = config.get("circuit_drawer", "text")
        if default_output == "auto":
            if _optionals.HAS_MATPLOTLIB:
                default_output = "mpl"
            else:
                default_output = "text"
        if wire_order is None:
            default_reverse_bits = config.get("circuit_reverse_bits", False)
    if output is None:
        output = default_output

    if reverse_bits is None:
        reverse_bits = default_reverse_bits

    if wire_order is not None and reverse_bits:
        raise VisualizationError(
            "The wire_order option cannot be set when the reverse_bits option is True."
        )

    complete_wire_order = wire_order
    if wire_order is not None:
        wire_order_len = len(wire_order)
        total_wire_len = circuit.num_qubits + circuit.num_clbits
        if wire_order_len not in [circuit.num_qubits, total_wire_len]:
            raise VisualizationError(
                f"The wire_order list (length {wire_order_len}) should as long as "
                f"the number of qubits ({circuit.num_qubits}) or the "
                f"total numbers of qubits and classical bits {total_wire_len}."
            )

        if len(set(wire_order)) != len(wire_order):
            raise VisualizationError("The wire_order list should not have repeated elements.")

        if wire_order_len == circuit.num_qubits:
            complete_wire_order = wire_order + list(range(circuit.num_qubits, total_wire_len))

    if (
        circuit.clbits
        and (reverse_bits or wire_order is not None)
        and not set(wire_order or []).issubset(set(range(circuit.num_qubits)))
    ):
        if cregbundle:
            warn(
                "cregbundle set to False since either reverse_bits or wire_order "
                "(over classical bit) has been set.",
                RuntimeWarning,
                2,
            )
        cregbundle = False

    def check_clbit_in_inst(circuit, cregbundle):
        if cregbundle is False:
            return False
        for inst in circuit.data:
            if isinstance(inst.operation, ControlFlowOp):
                for block in inst.operation.blocks:
                    if check_clbit_in_inst(block, cregbundle) is False:
                        return False
            elif inst.clbits and not isinstance(inst.operation, Measure):
                if cregbundle is not False:
                    warn(
                        "Cregbundle set to False since an instruction needs to refer"
                        " to individual classical wire",
                        RuntimeWarning,
                        3,
                    )
                return False

        return True

    cregbundle = check_clbit_in_inst(circuit, cregbundle)
    return {
        "circuit": circuit,
        "filename": filename,
        "reverse_bits": reverse_bits,
        "plot_barriers": plot_barriers,
        "justify": justify,
        "vertical_compression": vertical_compression,
        "idle_wires": idle_wires,
        "with_layout": with_layout,
        "fold": fold,
        "initial_state": initial_state,
        "cregbundle": cregbundle,
        "wire_order": complete_wire_order,
        "expr_len": expr_len,
        }
    # return _text_circuit_drawer(
    #     circuit,
    #     filename=filename,
    #     reverse_bits=reverse_bits,
    #     plot_barriers=plot_barriers,
    #     justify=justify,
    #     vertical_compression=vertical_compression,
    #     idle_wires=idle_wires,
    #     with_layout=with_layout,
    #     fold=fold,
    #     initial_state=initial_state,
    #     cregbundle=cregbundle,
    #     wire_order=complete_wire_order,
    #     expr_len=expr_len,
    # )

In [11]:
r"""Draw the quantum circuit. Use the output parameter to choose the drawing format:

    **text**: ASCII art TextDrawing that can be printed in the console.

    **mpl**: images with color rendered purely in Python using matplotlib.

    **latex**: high-quality images compiled via latex.

    **latex_source**: raw uncompiled latex output.

    .. warning::

        Support for :class:`~.expr.Expr` nodes in conditions and :attr:`.SwitchCaseOp.target`
        fields is preliminary and incomplete.  The ``text`` and ``mpl`` drawers will make a
        best-effort attempt to show data dependencies, but the LaTeX-based drawers will skip
        these completely.

    Args:
        circuit: The circuit to visualize.
        scale: Scale of image to draw (shrink if ``< 1.0``). Only used by
            the ``mpl``, ``latex`` and ``latex_source`` outputs. Defaults to ``1.0``.
        filename: File path to save image to. Defaults to ``None`` (result not saved in a file).
        style: Style name, file name of style JSON file, or a dictionary specifying the style.

            * The supported style names are ``"iqp"`` (default), ``"iqp-dark"``, ``"clifford"``,
                ``"textbook"`` and ``"bw"``.
            * If given a JSON file, e.g. ``my_style.json`` or ``my_style`` (the ``.json``
                extension may be omitted), this function attempts to load the style dictionary
                from that location. Note, that the JSON file must completely specify the
                visualization specifications. The file is searched for in
                ``qiskit/visualization/circuit/styles``, the current working directory, and
                the location specified in ``~/.qiskit/settings.conf``.
            * If a dictionary, every entry overrides the default configuration. If the
                ``"name"`` key is given, the default configuration is given by that style.
                For example, ``{"name": "textbook", "subfontsize": 5}`` loads the ``"texbook"``
                style and sets the subfontsize (e.g. the gate angles) to ``5``.
            * If ``None`` the default style ``"iqp"`` is used or, if given, the default style
                specified in ``~/.qiskit/settings.conf``.

        output: Select the output method to use for drawing the circuit.
            Valid choices are ``text``, ``mpl``, ``latex``, ``latex_source``.
            By default the `text` drawer is used unless the user config file
            (usually ``~/.qiskit/settings.conf``) has an alternative backend set
            as the default. For example, ``circuit_drawer = latex``. If the output
            kwarg is set, that backend will always be used over the default in
            the user config file.
        interactive: When set to ``True``, show the circuit in a new window
            (for ``mpl`` this depends on the matplotlib backend being used
            supporting this). Note when used with either the `text` or the
            ``latex_source`` output type this has no effect and will be silently
            ignored. Defaults to ``False``.
        reverse_bits: When set to ``True``, reverse the bit order inside
            registers for the output visualization. Defaults to ``False`` unless the
            user config file (usually ``~/.qiskit/settings.conf``) has an
            alternative value set. For example, ``circuit_reverse_bits = True``.
        plot_barriers: Enable/disable drawing barriers in the output
            circuit. Defaults to ``True``.
        justify: Options are ``left``, ``right`` or ``none``. If
            anything else is supplied, it defaults to left justified. It refers
            to where gates should be placed in the output circuit if there is
            an option. ``none`` results in each gate being placed in its own
            column.
        vertical_compression: ``high``, ``medium`` or ``low``. It
            merges the lines generated by the `text` output so the drawing
            will take less vertical room.  Default is ``medium``. Only used by
            the ``text`` output, will be silently ignored otherwise.
        idle_wires: Include idle wires (wires with no circuit elements)
            in output visualization. Default is ``True``.
        with_layout: Include layout information, with labels on the
            physical layout. Default is ``True``.
        fold: Sets pagination. It can be disabled using -1. In ``text``,
            sets the length of the lines. This is useful when the drawing does
            not fit in the console. If None (default), it will try to guess the
            console width using ``shutil.get_terminal_size()``. However, if
            running in jupyter, the default line length is set to 80 characters.
            In ``mpl``, it is the number of (visual) layers before folding.
            Default is 25.
        ax: Only used by the `mpl` backend. An optional ``matplotlib.axes.Axes``
            object to be used for the visualization output. If none is
            specified, a new matplotlib Figure will be created and used.
            Additionally, if specified there will be no returned Figure since
            it is redundant.
        initial_state: Adds :math:`|0\rangle` in the beginning of the qubit wires and
            :math:`0` to classical wires. Default is ``False``.
        cregbundle: If set to ``True``, bundle classical registers.
            Default is ``True``, except for when ``output`` is set to  ``"text"``.
        wire_order: A list of integers used to reorder the display
            of the bits. The list must have an entry for every bit with the bits
            in the range 0 to (``num_qubits`` + ``num_clbits``).
        expr_len: The number of characters to display if an :class:`~.expr.Expr`
            is used for the condition in a :class:`.ControlFlowOp`. If this number is exceeded,
            the string will be truncated at that number and '...' added to the end.

    Returns:
        :class:`.TextDrawing` or :class:`matplotlib.figure` or :class:`PIL.Image` or
        :class:`str`:

        * ``TextDrawing`` (if ``output='text'``)
            A drawing that can be printed as ascii art.
        * ``matplotlib.figure.Figure`` (if ``output='mpl'``)
            A matplotlib figure object for the circuit diagram.
        * ``PIL.Image`` (if ``output='latex``')
            An in-memory representation of the image of the circuit diagram.
        * ``str`` (if ``output='latex_source'``)
            The LaTeX source code for visualizing the circuit diagram.

    Raises:
        VisualizationError: when an invalid output method is selected
        ImportError: when the output methods requires non-installed libraries.

    Example:
        .. plot::
            :include-source:

            from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
            qc = QuantumCircuit(1, 1)
            qc.h(0)
            qc.measure(0, 0)
            qc.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})
    """

'Draw the quantum circuit. Use the output parameter to choose the drawing format:\n\n    **text**: ASCII art TextDrawing that can be printed in the console.\n\n    **mpl**: images with color rendered purely in Python using matplotlib.\n\n    **latex**: high-quality images compiled via latex.\n\n    **latex_source**: raw uncompiled latex output.\n\n    .. warning::\n\n        Support for :class:`~.expr.Expr` nodes in conditions and :attr:`.SwitchCaseOp.target`\n        fields is preliminary and incomplete.  The ``text`` and ``mpl`` drawers will make a\n        best-effort attempt to show data dependencies, but the LaTeX-based drawers will skip\n        these completely.\n\n    Args:\n        circuit: The circuit to visualize.\n        scale: Scale of image to draw (shrink if ``< 1.0``). Only used by\n            the ``mpl``, ``latex`` and ``latex_source`` outputs. Defaults to ``1.0``.\n        filename: File path to save image to. Defaults to ``None`` (result not saved in a file).\n    